In [3]:
NEUROGLANCER_ROOT = '/media/alexn/BstemAtlasDataBackup/neuroglancer_volumes/'

In [4]:
import socket
import neuroglancer
import subprocess
import numpy as np
from neuroglancer.server import global_server_args

class Neuroglancer_Viewer:
    def __init__(self, stack='MD585'):
        self.stack = stack
        self.local_volume_fp_root = './'
        
        neuroglancer.set_server_bind_address('0.0.0.0')
        global_server_args['bind_port'] = 80
        
        # Create viewer
        self.viewer = neuroglancer.Viewer()
        
        # Get the IP address
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(("8.8.8.8", 80))
        ip_name = s.getsockname()[0]
        s.close()

        # Removes the following symbols: ', ", [, ]
        self.url = str( 'http://' + ip_name + ':' + self.viewer.get_viewer_url().split(':')[2] )##Remote URL
        
    def set_local_volume_fp(self, fp):
        if fp[len(fp)-1]!='/':
            fp = fp+'/'
        self.local_volume_fp_root = fp
        
    def download_volumes(self):
        s3_root_fp = 's3://test-bucket-sid/alex_neuroglancer_volumes/'+self.stack+'/human_annotations_5um/'
        local_volume_fp = self.local_volume_fp_root+self.stack+'/human_annotations_5um/'
        
        command_list = ['aws','s3','cp','--recursive', s3_root_fp, local_volume_fp]
        subprocess.call( command_list )
    
    def add_stack(self):
        if self.stack == 'MD585':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    #source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD585_fullres')
                     source='precomputed://https://neuroglancer-data.s3.amazonaws.com/Neuroglancer_Images/Precomputed/MD585/MD585_prep2_raw_grayJpeg_precomputed')
        elif self.stack == 'MD589':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD589_fullres')
        elif self.stack == 'MD594':
            with self.viewer.txn() as s:
                s.layers[ self.stack+'_image'] = neuroglancer.ImageLayer(\
                    source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/MD594_fullres')
            
    def add_volume(self, colored=True):
        volume_fp = self.local_volume_fp_root+self.stack+'/human_annotation/solid_volume_5um/'
        if colored:
            volume_fn = 'volume_colored.npy'
            color_segments=[]
            for i in range(1,50):
                color_segments.append(i)
        else:
            volume_fn = 'volume.npy'
        
        xy_ng_resolution_um = 5
        volume_data = np.load( volume_fp+volume_fn )
        
        with self.viewer.txn() as s:
            s.layers[ self.stack+"_Annotations" ] = neuroglancer.SegmentationLayer(
                source = neuroglancer.LocalVolume(
                    data=volume_data, # Z,Y,X
                    voxel_size=[ xy_ng_resolution_um*1000, xy_ng_resolution_um*1000, 20000], # X Y Z
                    voxel_offset = [ 0, 0, 0] # X Y Z
                ),
                segments = color_segments
            )
            
    def reset_orientation(self):
        with self.viewer.txn() as s:
            # Resets X/Y/Z plane orientation
            s.navigation.pose.orientation = [0,0,0,1]
            # Zooms out 
            s.navigation.zoomFactor = 5000 # 5000 If xy, 10000 If 4panel
            # Resets 3D Viewer Orientation
            s.perspectiveOrientation = [0,0,0,1]
            # Zooms out
            s.perspectiveZoom = 75000

            s.layout = 'xy' # '3d'/'4panel'/'xy'
            
    def stop(self):
        neuroglancer.stop()

In [8]:
viewers = []
print 'MD585'
viewers.append( Neuroglancer_Viewer(stack='MD585') )
viewers[0].add_stack()
#viewers[0].set_local_volume_fp( fp=NEUROGLANCER_ROOT )
#viewers[0].add_volume()
viewers[0].reset_orientation()
print viewers[0].url

# print 'MD589'
# viewers.append( Neuroglancer_Viewer(stack='MD589') )
# viewers[1].add_stack()
# viewers[1].set_local_volume_fp( fp=NEUROGLANCER_ROOT )
# viewers[1].add_volume()
# viewers[1].reset_orientation()
# print viewers[1].url

# print 'MD594'
# viewers.append( Neuroglancer_Viewer(stack='MD594') )
# viewers[2].add_stack()
# viewers[2].set_local_volume_fp( fp=NEUROGLANCER_ROOT )
# viewers[2].add_volume()
# viewers[2].reset_orientation()
# print viewers[2].url

MD585
http://132.239.73.85:43625/v/29ca6337331b6e6f8125090cc142227f7651d9fc/


In [7]:
viewers[0].stop()
del viewers[0]
viewers[0].stop()
del viewers[0]
viewers[0].stop()
del viewers[0]

IndexError: list index out of range

In [3]:
new_viewer = Neuroglancer_Viewer(stack='MD594') # 8 off
new_viewer.add_stack()
new_viewer.reset_orientation()

print new_viewer.url

http://132.239.73.85:42849/v/b6486a96bca246b911bfeb7a6647331877432ce7/


In [3]:
new_viewer = Neuroglancer_Viewer(stack='MD589') # 8 off
new_viewer.add_stack()
new_viewer.reset_orientation()

print new_viewer.url

http://132.239.73.85:42211/v/e61617fb1f4f04f4e8e9310c73b1deeb0ce4dd25/


In [3]:
new_viewer = Neuroglancer_Viewer(stack='MD585')
new_viewer.add_stack()

new_viewer.set_local_volume_fp( fp=NEUROGLANCER_ROOT )
new_viewer.add_volume()

new_viewer.reset_orientation()

print new_viewer.url

http://132.239.73.85:32961/v/a78262516d1329bc39c62a444903a6fdeb007c50/


In [4]:
new_viewer.set_local_volume_fp( fp=NEUROGLANCER_ROOT )
new_viewer.add_volume()
new_viewer.reset_orientation()

In [17]:
new_viewer.stop()
del new_viewer

In [57]:
import subprocess
s3_root_fp = 's3://test-bucket-sid/alex_neuroglancer_volumes/'+'MD585'+'/human_annotations_5um'
        
command_list = ['aws','s3','cp','--recursive',s3_root_fp, '/home/alexn/Desktop/test_ng/']
subprocess.call( command_list )

1

In [41]:
! rm -r /home/alexn/test_ng

In [ ]:
precomputed://https://neuroglancer-data.s3.amazonaws.com/Neuroglancer_Images/Precomputed/MD585/MD585_prep2_raw_grayJpeg_precomputed

In [ ]:
precomputed://https://neuroglancer-data.s3.amazonaws.com/Neuroglancer_Images/Precomputed/MD589/MD589_prep2_raw_grayJpeg_precomputed

In [ ]:
# %7B : {
# %7D : }
# %5B : [
# %5C : 
# %5D : ]
# %2C : ,

# Copy and paste the below cell!

In [ ]:
def Neuroglancer_Json_Constructor():
    def __init__(self, stack):
        init_json = {}

https://neuroglancer-demo.appspot.com/#!
{
  "layers": [
    {
      "source": "precomputed://https://neuroglancer-data.s3.amazonaws.com/Neuroglancer_Images/Precomputed/MD585/MD585_prep2_raw_grayJpeg_precomputed",
      "type": "image",
      "name": "image"
    },
    {
      "source": "precomputed://https://neuroglancer-data.s3.amazonaws.com/Neuroglancer_Images/Precomputed/MD589/MD589_prep2_raw_grayJpeg_precomputed",
      "type": "image",
      "name": "image_2"
    },
    {
      "tab": "transform",
      "source": "precomputed://https://test-bucket-sid.s3.amazonaws.com/final_precomputed_volumes/MD589",
      "type": "segmentation",
      "skeletonRendering": {
        "mode2d": "lines_and_points",
        "mode3d": "lines"
      },
      "name": "annotation"
    }
  ],
  "navigation": {
    "pose": {
      "position": {
        "voxelSize": [
          460,
          460,
          20000
        ],
        "voxelCoordinates": [
          9745,
          6531,
          170.47999572753906
        ]
      }
    },
    "zoomFactor": 14720
  },
  "perspectiveOrientation": [
    0.0019906109664589167,
    -0.0022145232651382685,
    0.007156445644795895,
    0.9999699592590332
  ],
  "perspectiveZoom": 131072,
  "selectedLayer": {
    "layer": "annotation",
    "visible": true
  },
  "layout": "4panel"
}

In [1]:
from __future__ import print_function

import numpy as np

import neuroglancer

neuroglancer.set_server_bind_address('127.0.0.1')

#neuroglancer.set_static_content_source(url=args.static_content_url)

a = np.zeros((3, 100, 100, 100), dtype=np.uint8)
ix, iy, iz = np.meshgrid(* [np.linspace(0, 1, n) for n in a.shape[1:]], indexing='ij')
a[0, :, :, :] = np.abs(np.sin(4 * (ix + iy))) * 255
a[1, :, :, :] = np.abs(np.sin(4 * (iy + iz))) * 255
a[2, :, :, :] = np.abs(np.sin(4 * (ix + iz))) * 255

b = np.cast[np.uint32](np.floor(np.sqrt((ix - 0.5)**2 + (iy - 0.5)**2 + (iz - 0.5)**2) * 10))
b = np.pad(b, 1, 'constant')

viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.voxel_size = [10, 10, 10]
    s.layers.append(
        name='a',
        layer=neuroglancer.LocalVolume(
            data=a,
            # offset is in nm, not voxels
            offset=(200, 300, 150),
            voxel_size=s.voxel_size,
        ),
        shader="""
void main() {
  emitRGB(vec3(toNormalized(getDataValue(0)),
               toNormalized(getDataValue(1)),
               toNormalized(getDataValue(2))));
}
""")
    s.layers.append(
        name='b', layer=neuroglancer.LocalVolume(
            data=b,
            voxel_size=s.voxel_size,
        ))

print(viewer)

http://127.0.0.1:41627/v/a03192aafab922ff75c364f6d4916934691f392a/


In [ ]:
https://neuroglancer-demo.appspot.com/#!
{
  "layers": [
    {
      "source": "precomputed://https://neuroglancer-data.s3.amazonaws.com/Neuroglancer_Images/Precomputed/MD585/MD585_prep2_raw_grayJpeg_precomputed",
      "type": "image",
      "shader": "\nvoid main() {\n  emitRGB(vec3(toNormalized(getDataValue(1)),\n               toNormalized(getDataValue(1)),\n               toNormalized(getDataValue(2))));\n}\n",
      "name": "a"
    }
  ],
  "navigation": {
    "pose": {
      "position": {
        "voxelSize": [
          10,
          10,
          10
        ],
        "voxelCoordinates": [
          64.56402587890625,
          77.08424377441406,
          65
        ]
      }
    },
    "zoomFactor": 10
  },
  "perspectiveOrientation": [
    -0.6836387515068054,
    -0.17720557749271393,
    0.6858879327774048,
    0.175482377409935
  ],
  "perspectiveZoom": 4.154038645375808,
  "selectedLayer": {
    "layer": "a",
    "visible": true
  },
  "layout": "4panel"
}